In [8]:
import pandas as pd
import yfinance as yf
from backtesting import Backtest
from src.strategies import MomentumTimeSeries, SmaCross, MeanReversionLongOnly
from src.utils import load_data

In [9]:
ticker = "IAG.L"
period = "10y"
stock = yf.Ticker(ticker.upper())
data = stock.history(period=period)

/Users/ivankyulev/Projects/Notebooks/.venv/lib/python3.11/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


In [10]:
bt = Backtest(data, SmaCross, cash=10_000, commission=0.004, trade_on_close=False)
stats = bt.run()
stats

Start                     2014-03-07 00:00...
End                       2024-03-07 00:00...
Duration                   3653 days 00:00:00
Exposure Time [%]                    80.31746
Equity Final [$]                  4647.157522
Equity Peak [$]                  16425.543792
Return [%]                         -53.528425
Buy & Hold Return [%]              -51.805188
Return (Ann.) [%]                   -7.377022
Volatility (Ann.) [%]               37.758618
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -81.270796
Avg. Drawdown [%]                   -8.636339
Max. Drawdown Duration     1718 days 00:00:00
Avg. Drawdown Duration      108 days 00:00:00
# Trades                                   11
Win Rate [%]                        45.454545
Best Trade [%]                       29.27564
Worst Trade [%]                     -57.44799
Avg. Trade [%]                    

In [6]:
bt.plot()

/Users/ivankyulev/Projects/Notebooks/.venv/lib/python3.11/site-packages/backtesting/_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')


Row(id='1450', ...)

In [17]:
%%time
stats = bt.optimize(n1=range(5, 30, 5),
                    n2=range(10, 70, 5),
                    maximize='Equity Final [$]',
                    constraint=lambda param: param.n1 < param.n2)
stats

/Users/ivankyulev/Projects/Notebooks/.venv/lib/python3.11/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


AttributeError: Strategy 'MomentumTimeSeries' is missing parameter 'n1'.Strategy class should define parameters as class variables before they can be optimized or run with.

In [16]:
bt.plot()

/Users/ivankyulev/Projects/Notebooks/.venv/lib/python3.11/site-packages/backtesting/_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')


Row(id='3210', ...)

In [12]:
%%time
bt = Backtest(data, MomentumTimeSeries, cash=10_000, commission=0, trade_on_close=False)
stats = bt.optimize(
    lookback=range(1,10,1),
    maximize='Equity Final [$]',
    # constraint=lambda param: param,
)
stats

/Users/ivankyulev/Projects/Notebooks/.venv/lib/python3.11/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


CPU times: user 3.53 s, sys: 17 ms, total: 3.55 s
Wall time: 3.55 s


Start                     2014-03-07 00:00...
End                       2024-03-07 00:00...
Duration                   3653 days 00:00:00
Exposure Time [%]                   99.563492
Equity Final [$]                 17888.583402
Equity Peak [$]                  31131.753988
Return [%]                          78.885834
Buy & Hold Return [%]              -51.805188
Return (Ann.) [%]                    5.988219
Volatility (Ann.) [%]               53.180473
Sharpe Ratio                         0.112602
Sortino Ratio                        0.212319
Calmar Ratio                          0.09951
Max. Drawdown [%]                   -60.17707
Avg. Drawdown [%]                  -20.442811
Max. Drawdown Duration     1929 days 00:00:00
Avg. Drawdown Duration      226 days 00:00:00
# Trades                                 2509
Win Rate [%]                        48.465524
Best Trade [%]                      33.585662
Worst Trade [%]                    -15.332456
Avg. Trade [%]                    

In [15]:
bt.plot()

/Users/ivankyulev/Projects/Notebooks/.venv/lib/python3.11/site-packages/backtesting/_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')


Row(id='2276', ...)

In [14]:
bt = Backtest(data, MomentumTimeSeries, cash=10_000, commission=0, trade_on_close=True)
stats = bt.run()
stats

Start                     2014-03-07 00:00...
End                       2024-03-07 00:00...
Duration                   3653 days 00:00:00
Exposure Time [%]                   99.801587
Equity Final [$]                  5852.055458
Equity Peak [$]                  15736.133194
Return [%]                         -41.479445
Buy & Hold Return [%]              -51.805188
Return (Ann.) [%]                   -5.216914
Volatility (Ann.) [%]               44.892849
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -74.02291
Avg. Drawdown [%]                  -13.125982
Max. Drawdown Duration     2658 days 00:00:00
Avg. Drawdown Duration      202 days 00:00:00
# Trades                                 2515
Win Rate [%]                        48.031809
Best Trade [%]                      30.818739
Worst Trade [%]                    -22.537888
Avg. Trade [%]                    